In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Load the datasets
disaster_decs = pd.read_csv('https://raw.githubusercontent.com/pauladel/DS3001-Project/refs/heads/main/raw-data/us_disaster_declarations.csv')
fema_funding = pd.read_parquet('https://www.fema.gov/api/open/v2/EmergencyManagementPerformanceGrants.parquet')

# Preprocessing disaster_decs dataset
# Convert the fy_declared column to integer if not already
disaster_decs['fy_declared'] = disaster_decs['fy_declared'].astype(int)

# Count the frequency of each disaster type per state per year
disaster_frequency = disaster_decs.groupby(['state', 'fy_declared', 'incident_type']).size().reset_index(name='disaster_count')

disaster_frequency.head()


,state,fy_declared,incident_type,disaster_count
0,AK,1954,Other,1
1,AK,1955,Other,1
2,AK,1956,Other,1
3,AK,1964,Earthquake,1
4,AK,1967,Flood,1


In [ ]:
# Preprocessing fema_funding dataset
# Ensure projectStartDate is of string type
fema_funding['projectStartDate'] = fema_funding['projectStartDate'].astype(str)

# Extract year from projectStartDate
fema_funding['year'] = fema_funding['projectStartDate'].str[:4].astype(int)

# Group by state and year to get total funding amount requested
funding_summary = fema_funding.groupby(['state', 'year'])['fundingAmount'].sum().reset_index()

funding_summary.head()


,state,year,fundingAmount
0,Alabama,2014,5795991.0
1,Alabama,2015,5780028.0
2,Alabama,2016,5762090.0
3,Alabama,2019,5728503.0
4,Alabama,2020,5810021.0


In [ ]:
reversed_state_mapping = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR',
    'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE',
    'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
    'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
    'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV',
    'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY',
    'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT',
    'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV',
    'Wisconsin': 'WI', 'Wyoming': 'WY'
}

funding_summary['state'] = funding_summary['state'].map(reversed_state_mapping)

funding_summary.head()

,state,year,fundingAmount
0,AL,2014,5795991.0
1,AL,2015,5780028.0
2,AL,2016,5762090.0
3,AL,2019,5728503.0
4,AL,2020,5810021.0


In [ ]:
# Merge the two datasets on state and year
merged_data = pd.merge(
    disaster_pivot,
    funding_summary,
    left_on=['state', 'fy_declared'],
    right_on=['state', 'year'],
    how='inner'
)

merged_data



,state,fy_declared,Biological,Chemical,Coastal Storm,Dam/Levee Break,Drought,Earthquake,Fire,Fishing Losses,...,Terrorist,Tornado,Toxic Substances,Tropical Storm,Tsunami,Typhoon,Volcanic Eruption,Winter Storm,year,fundingAmount
0,AK,2013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2013,2425109.67
1,AK,2014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014,2989528.98
2,AK,2015,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015,796939.35
3,AK,2016,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016,2182115.22
4,AK,2018,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018,3115502.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,WY,2012,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2012,124800.00
336,WY,2015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015,57000.00
337,WY,2016,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016,3001110.91
338,WY,2018,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018,87390.29


In [ ]:
# Drop the redundant year column
merged_data.drop('year', axis=1, inplace=True)

# Define features (X) and target (y)
X = merged_data.drop(columns=['state', 'fy_declared', 'fundingAmount'])
y = merged_data['fundingAmount']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

# Optional: Display the coefficients
coefficients = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': model.coef_
})
print(coefficients.sort_values(by='Coefficient', ascending=False))

Mean Squared Error: 75236351229713.56
R-squared: -1.6420160847190273
              Feature   Coefficient
13              Other  7.458012e+06
5          Earthquake  6.578130e+06
19   Toxic Substances  3.527077e+06
3     Dam/Levee Break  2.175235e+06
6                Fire  2.714233e+05
18            Tornado  6.178786e+04
8               Flood  4.682506e+04
14   Severe Ice Storm  3.367254e+04
11          Hurricane  2.696142e+04
0          Biological  8.313693e+03
15       Severe Storm  1.473953e+03
1            Chemical  2.997695e-09
10        Human Cause  9.313226e-10
9            Freezing  1.164153e-10
22            Typhoon  0.000000e+00
17          Terrorist  0.000000e+00
23  Volcanic Eruption  0.000000e+00
20     Tropical Storm  0.000000e+00
21            Tsunami  0.000000e+00
7      Fishing Losses -4.656613e-10
4             Drought -2.153683e-09
16          Snowstorm -4.004818e+04
24       Winter Storm -1.177787e+05
2       Coastal Storm -1.715085e+05
12      Mud/Landslide -3.614730

Cite: Chat GPT, FEMA Datasets